# Permuted Pixel MNIST Demo 
Light weighted demo of our DilatedRNN on Pixel MNist with permutation.

In [1]:
import sys
sys.path.append("./models")
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from classification_models import drnn_classification

In [2]:
# configurations
data_dir = "./MNIST_data"
n_steps = 28*28
input_dims = 1
n_classes = 10 

# model config
cell_type = "GRU"
assert(cell_type in ["RNN", "LSTM", "GRU"])
hidden_structs = [20, 30, 40, 50]
dilations = [4, 8, 16, 32]
assert(len(hidden_structs) == len(dilations))

# learning config
batch_size = 128
learning_rate = 1.0e-3
training_iters = batch_size * 30000
testing_step = 5000
display_step = 100

# permutation seed 
seed = 92916

In [3]:
mnist = input_data.read_data_sets(data_dir, one_hot=True)
if 'seed' in globals():
    rng_permute = np.random.RandomState(seed)
    idx_permute = rng_permute.permutation(n_steps)
else:
    idx_permute = np.random.permutation(n_steps)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
# build computation graph
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, n_steps, input_dims])
y = tf.placeholder(tf.float32, [None, n_classes])    
global_step = tf.Variable(0, name='global_step', trainable=False)

# build prediction graph
print ("==> Building a dRNN with %s cells" %cell_type)
pred = drnn_classification(x, hidden_structs, dilations, n_steps, n_classes, input_dims, cell_type)

# build loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# evaluation model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

==> Building a dRNN with GRU cells
Building layer: multi_dRNN_dilation_4, input length: 784, dilation rate: 4, input dim: 1.
=====> Input length for sub-RNN: 196
Building layer: multi_dRNN_dilation_8, input length: 784, dilation rate: 8, input dim: 20.
=====> Input length for sub-RNN: 98
Building layer: multi_dRNN_dilation_16, input length: 784, dilation rate: 16, input dim: 30.
=====> Input length for sub-RNN: 49
Building layer: multi_dRNN_dilation_32, input length: 784, dilation rate: 32, input dim: 40.
=====> 16 time points need to be padded. 
=====> Input length for sub-RNN: 25


In [5]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [6]:
step = 0
train_results = []
validation_results = []
test_results = []

while step * batch_size < training_iters:
    batch_x, batch_y = mnist.train.next_batch(batch_size)    
    batch_x = batch_x[:, idx_permute]
    batch_x = batch_x.reshape([batch_size, n_steps, input_dims])

    feed_dict = {
        x : batch_x,
        y : batch_y
    }
    cost_, accuracy_, step_,  _ = sess.run([cost, accuracy, global_step, optimizer], feed_dict=feed_dict)    
    train_results.append((step_, cost_, accuracy_))    

    if (step + 1) % display_step == 0:
        print ("Iter " + str(step + 1) + ", Minibatch Loss: " + "{:.6f}".format(cost_) \
        + ", Training Accuracy: " + "{:.6f}".format(accuracy_))
             
    if (step + 1) % testing_step == 0:
        
        # validation performance
        batch_x = mnist.validation.images
        batch_y = mnist.validation.labels

        # permute the data
        batch_x = batch_x[:, idx_permute]        
        batch_x = batch_x.reshape([-1, n_steps, input_dims])
        feed_dict = {
            x : batch_x,
            y : batch_y
        }
        cost_, accuracy__, step_ = sess.run([cost, accuracy, global_step], feed_dict=feed_dict)
        validation_results.append((step_, cost_, accuracy__))
        
        # test performance
        batch_x = mnist.test.images
        batch_y = mnist.test.labels
        batch_x = batch_x[:, idx_permute]        
        batch_x = batch_x.reshape([-1, n_steps, input_dims])
        feed_dict = {
            x : batch_x,
            y : batch_y
        }
        cost_, accuracy_, step_ = sess.run([cost, accuracy, global_step], feed_dict=feed_dict)
        test_results.append((step_, cost_, accuracy_))        
        print ("========> Validation Accuarcy: " + "{:.6f}".format(accuracy__) \
        + ", Testing Accuarcy: " + "{:.6f}".format(accuracy_)) 
    step += 1

Iter 100, Minibatch Loss: 1.653754, Training Accuracy: 0.437500
Iter 200, Minibatch Loss: 1.142690, Training Accuracy: 0.617188
Iter 300, Minibatch Loss: 0.894196, Training Accuracy: 0.671875
Iter 400, Minibatch Loss: 1.021729, Training Accuracy: 0.664062
Iter 500, Minibatch Loss: 1.125893, Training Accuracy: 0.710938
Iter 600, Minibatch Loss: 1.037349, Training Accuracy: 0.632812
Iter 700, Minibatch Loss: 0.785295, Training Accuracy: 0.773438
Iter 800, Minibatch Loss: 0.841199, Training Accuracy: 0.750000
Iter 900, Minibatch Loss: 0.926831, Training Accuracy: 0.710938
Iter 1000, Minibatch Loss: 0.729665, Training Accuracy: 0.781250
Iter 1100, Minibatch Loss: 0.842229, Training Accuracy: 0.750000
Iter 1200, Minibatch Loss: 0.677812, Training Accuracy: 0.828125
Iter 1300, Minibatch Loss: 0.663875, Training Accuracy: 0.820312
Iter 1400, Minibatch Loss: 0.546523, Training Accuracy: 0.828125
Iter 1500, Minibatch Loss: 0.584997, Training Accuracy: 0.789062
Iter 1600, Minibatch Loss: 0.57922

Iter 12500, Minibatch Loss: 0.215983, Training Accuracy: 0.953125
Iter 12600, Minibatch Loss: 0.221987, Training Accuracy: 0.929688
Iter 12700, Minibatch Loss: 0.103629, Training Accuracy: 0.960938
Iter 12800, Minibatch Loss: 0.128298, Training Accuracy: 0.953125
Iter 12900, Minibatch Loss: 0.117450, Training Accuracy: 0.960938
Iter 13000, Minibatch Loss: 0.254722, Training Accuracy: 0.914062
Iter 13100, Minibatch Loss: 0.109978, Training Accuracy: 0.960938
Iter 13200, Minibatch Loss: 0.112591, Training Accuracy: 0.960938
Iter 13300, Minibatch Loss: 0.209530, Training Accuracy: 0.945312
Iter 13400, Minibatch Loss: 0.186092, Training Accuracy: 0.953125
Iter 13500, Minibatch Loss: 0.144550, Training Accuracy: 0.937500
Iter 13600, Minibatch Loss: 0.156939, Training Accuracy: 0.976562
Iter 13700, Minibatch Loss: 0.127148, Training Accuracy: 0.937500
Iter 13800, Minibatch Loss: 0.069949, Training Accuracy: 0.968750
Iter 13900, Minibatch Loss: 0.082443, Training Accuracy: 0.968750
Iter 14000

Iter 24800, Minibatch Loss: 0.023974, Training Accuracy: 1.000000
Iter 24900, Minibatch Loss: 0.086616, Training Accuracy: 0.968750
Iter 25000, Minibatch Loss: 0.073173, Training Accuracy: 0.968750
========> Validation Accuarcy: 0.948600, Testing Accuarcy: 0.948500
Iter 25100, Minibatch Loss: 0.080487, Training Accuracy: 0.968750
Iter 25200, Minibatch Loss: 0.049912, Training Accuracy: 0.976562
Iter 25300, Minibatch Loss: 0.130959, Training Accuracy: 0.945312
Iter 25400, Minibatch Loss: 0.066505, Training Accuracy: 0.968750
Iter 25500, Minibatch Loss: 0.039057, Training Accuracy: 0.976562
Iter 25600, Minibatch Loss: 0.044169, Training Accuracy: 0.984375
Iter 25700, Minibatch Loss: 0.129876, Training Accuracy: 0.953125
Iter 25800, Minibatch Loss: 0.115520, Training Accuracy: 0.945312
Iter 25900, Minibatch Loss: 0.125561, Training Accuracy: 0.960938
Iter 26000, Minibatch Loss: 0.108634, Training Accuracy: 0.976562
Iter 26100, Minibatch Loss: 0.135492, Training Accuracy: 0.945312
Iter 262